# Agents

Compared to chains, agents are more flexible, actions aren't hard coded. With Agents, a LLM is used as a reasoning engine to determine which actions to take and in which order.

Some important keywords:
 - **AgentAction**: dataclass that is an action. Has a tool property and `tool_input``.
 - **AgentFinish**: dataclass that indicates the agent is done and will return to the user. Has `return_values`` dictionary. Most of the time, it will have 1 value for the key `output`
 - **intermediate_steps**: a list of AgentActions that are executed before the AgentFinish is returned to the user.


## Agent

Chain responsible for deciding what stept to take next:
The inputs to this chain are:
  - list of available tools
  - user input
  - any previously executed steps

## Tools

Tools are functions that an agent calls. There are 2 considirations:
 - Giving the agent access to the right tools
 - Describing the tools in a way that is helpful for the agent

## Toolkits

Set of tools an agent has access to, generally 3-5 tools in a toolkit.

## AgentExecutor

Runtime for an agent, this is what calls the agent and executes the actions it chooses.

You do have other types of agent runtimes:
 - Plan-and-execute Agent
 - Baby AGI
 - Auto GPT


## Building your own agent

In [2]:
# load the LLM to control the agent
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

# define some tools
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

# Now let us create the prompt. Because OpenAI Function Calling is finetuned for tool usage, 
# we hardly need any instructions on how to reason, or how to output format. 
# We will just have two input variables: input (for the user question) and agent_scratchpad (for any previous steps taken)

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# How does the agent know what tools it can use? Those are passed in as a separate argument, 
# so we can bind those as keyword arguments to the LLM.

from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

# Putting those pieces together, we can now create the agent.
# We import two util functions for formmating intermediate steps + converting the output to an agent finish

from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()





AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_word_length', 'arguments': '{\n  "word": "educa"\n}'}})])

In [ ]:
# Now we can use the agent to answer questions. Let us start with a simple one.

agent.invoke({
    "input": "how many letters in the word educa?",
    "intermediate_steps": []
})

# We can see that it responds with an AgentAction to take (it's actually an AgentActionMessageLog 
# a subclass of AgentAction which also tracks the full message log).

In [3]:
# So this is just the first step - now we need to write a runtime for this. 
# The simplest one is just one that continuously loops, calling the agent, 
# then taking the action, and repeating until an AgentFinish is returned.
# Let's code that up below:

from langchain.schema.agent import AgentFinish
intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "how many letters in the word educa?",
        "intermediate_steps": intermediate_steps
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        tool = {
            "get_word_length": get_word_length
        }[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

get_word_length {'word': 'educa'}
There are 5 letters in the word "educa".


In [4]:
# You can simplify this with AgentExecutor:

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "how many letters in the word educa?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'how many letters in the word educa?',
 'output': 'There are 5 letters in the word "educa".'}

### Now with memory!

In [5]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# We can then set up a list to track the chat history

from langchain.schema.messages import HumanMessage, AIMessage
chat_history = []

# We can then put it all together!

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# When running, we now need to track the inputs and outputs as chat history

input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.append(HumanMessage(content=input1))
chat_history.append(AIMessage(content=result['output']))
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}